# PySpark Practice Exercises

**Author:** Benjamin Gao  
**Date:** October 27, 2025  
**Purpose:** Hands-on practice with PySpark DataFrames

---

## 📋 Exercise Overview

This notebook contains 6 comprehensive exercises:

1. **Create & Read CSV** - Manually create CSV with customer data
2. **Filter Data** - Find customers who bought more than 5 items
3. **Add Columns** - Calculate TotalCost (Quantity × Price)
4. **Handle Missing Values** - Fill with averages and constants
5. **Aggregations** - Group by Product and calculate statistics
6. **Joins** - Join with Product details using inner, left, right joins

---

## 🔧 Setup: Environment Configuration

Run this cell first to configure Java and import libraries.

In [ ]:
# Setup: JAVA_HOME + Import Libraries
import os
import sys

# Set JAVA_HOME (adjust if needed)
java_home = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
os.environ["JAVA_HOME"] = java_home

# Import PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

print(f"✅ JAVA_HOME: {java_home}")
print(f"✅ Python: {sys.executable}")
print("✅ Libraries imported successfully!")

In [ ]:
# Create SparkSession
spark = (
    SparkSession.builder
        .master("local[1]")
        .appName("PySparkExercise")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .config("spark.ui.enabled", "false")
        .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark {spark.version} is running!")
print(f"✅ Master: {spark.sparkContext.master}")

---

## Exercise 1: Create CSV File and Read into DataFrame

**Task:** 
1. Manually create a CSV file with columns: `CustomerID`, `Product`, `Quantity`, `Price`
2. Read the CSV file into a PySpark DataFrame

**Expected Output:** DataFrame with customer purchase data

In [ ]:
# Step 1: Create CSV data
csv_data = """CustomerID,Product,Quantity,Price
C001,Laptop,2,1200.00
C002,Mouse,10,25.50
C003,Keyboard,5,80.00
C004,Monitor,3,300.00
C001,Mouse,8,25.50
C002,Laptop,1,1200.00
C005,Keyboard,,80.00
C003,Monitor,4,
C006,Laptop,6,1200.00
C007,Mouse,12,25.50
C004,Keyboard,3,80.00
C008,Monitor,,300.00
C005,Mouse,15,25.50
C009,Laptop,1,1200.00
C010,Keyboard,7,80.00"""

# Step 2: Write to CSV file
with open("customer_purchases.csv", "w") as f:
    f.write(csv_data)

print("✅ CSV file 'customer_purchases.csv' created!")

# Step 3: Read CSV into DataFrame
df = spark.read.csv("customer_purchases.csv", header=True, inferSchema=True)

print("\n📊 Customer Purchases DataFrame:")
df.show()

print("\n📋 Schema:")
df.printSchema()

---

## Exercise 2: Filter Customers Who Bought More Than 5 Items

**Task:** Filter the DataFrame to show only customers who purchased more than 5 items (Quantity > 5)

**Expected Output:** DataFrame with high-quantity purchases

In [ ]:
# Filter: Quantity > 5
df_filtered = df.filter(df.Quantity > 5)

print("🔍 Customers who bought MORE than 5 items:")
df_filtered.show()

print(f"\n📊 Total records: {df_filtered.count()}")

---

## Exercise 3: Add TotalCost Column

**Task:** Add a new column `TotalCost` calculated as `Quantity × Price`

**Expected Output:** DataFrame with TotalCost column

In [ ]:
# Add TotalCost column
df_with_cost = df.withColumn("TotalCost", col("Quantity") * col("Price"))

print("💰 DataFrame with TotalCost:")
df_with_cost.show()

print("\n📋 Updated Schema:")
df_with_cost.printSchema()

---

## Exercise 4: Handle Missing Values

**Task:** 
1. Fill missing `Quantity` with the **average** Quantity
2. Fill missing `Price` with a **constant** value (e.g., 50.0)

**Expected Output:** DataFrame with no missing values

In [ ]:
# Check missing values before
print("❌ Missing values BEFORE handling:")
df_with_cost.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in df_with_cost.columns]
).show()

# Calculate average Quantity (for non-null values)
avg_quantity = df_with_cost.select(avg("Quantity")).first()[0]
print(f"\n📊 Average Quantity: {avg_quantity:.2f}")

# Fill missing values
df_filled = df_with_cost.fillna({
    "Quantity": avg_quantity,  # Fill with average
    "Price": 50.0               # Fill with constant
})

# Recalculate TotalCost after filling
df_filled = df_filled.withColumn("TotalCost", col("Quantity") * col("Price"))

print("\n✅ DataFrame AFTER filling missing values:")
df_filled.show()

# Verify no missing values
print("\n✅ Missing values AFTER handling:")
df_filled.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in df_filled.columns]
).show()

---

## Exercise 5: Group by Product and Calculate Aggregations

**Task:** Group by `Product` and calculate:
- Average TotalCost
- Sum of TotalCost
- Min TotalCost
- Max TotalCost
- Count of records

**Expected Output:** Aggregated statistics per product

In [ ]:
# Group by Product and aggregate
df_aggregated = df_filled.groupBy("Product").agg(
    avg("TotalCost").alias("Avg_TotalCost"),
    sum("TotalCost").alias("Sum_TotalCost"),
    min("TotalCost").alias("Min_TotalCost"),
    max("TotalCost").alias("Max_TotalCost"),
    count("*").alias("Record_Count")
).orderBy("Product")

print("📊 Product Statistics:")
df_aggregated.show()

# Optional: Show with formatted numbers
print("\n💰 Formatted Statistics:")
df_aggregated.select(
    col("Product"),
    format_number("Avg_TotalCost", 2).alias("Avg ($)"),
    format_number("Sum_TotalCost", 2).alias("Total ($)"),
    format_number("Min_TotalCost", 2).alias("Min ($)"),
    format_number("Max_TotalCost", 2).alias("Max ($)"),
    col("Record_Count").alias("Count")
).show(truncate=False)

---

## Exercise 6: Join with Product Details

**Task:** 
1. Create another CSV file with Product details (ProductID, Category)
2. Perform **Inner**, **Left**, and **Right** joins

**Expected Output:** Three different join results

In [ ]:
# Step 1: Create Product Details CSV
product_csv = """ProductID,Category
Laptop,Electronics
Mouse,Electronics
Keyboard,Electronics
Monitor,Electronics
Webcam,Electronics"""

with open("product_details.csv", "w") as f:
    f.write(product_csv)

print("✅ CSV file 'product_details.csv' created!")

# Step 2: Read Product Details into DataFrame
df_products = spark.read.csv("product_details.csv", header=True, inferSchema=True)

print("\n📦 Product Details:")
df_products.show()

In [ ]:
# Rename column for clarity (Product → ProductID in df_filled)
# Note: We'll join on Product name

# INNER JOIN: Only matching products
print("🔗 INNER JOIN (only products in BOTH tables):")
df_inner = df_filled.join(
    df_products, 
    df_filled.Product == df_products.ProductID, 
    how="inner"
).drop("ProductID")  # Remove duplicate column

df_inner.show()

print(f"Records: {df_inner.count()}")

In [ ]:
# LEFT JOIN: All customer purchases + matching product details
print("🔗 LEFT JOIN (all purchases, with product details if available):")
df_left = df_filled.join(
    df_products, 
    df_filled.Product == df_products.ProductID, 
    how="left"
).drop("ProductID")

df_left.show()

print(f"Records: {df_left.count()}")

In [ ]:
# RIGHT JOIN: All products + matching customer purchases
print("🔗 RIGHT JOIN (all products, with purchases if available):")
df_right = df_filled.join(
    df_products, 
    df_filled.Product == df_products.ProductID, 
    how="right"
).drop("ProductID")

df_right.show()

print(f"Records: {df_right.count()}")

# Notice: "Webcam" appears with NULL values (no customer bought it)

---

## 🎯 Summary

### What You've Practiced:

✅ **Exercise 1:** Created CSV and read into DataFrame  
✅ **Exercise 2:** Filtered data with conditions  
✅ **Exercise 3:** Added computed columns  
✅ **Exercise 4:** Handled missing values (avg & constant)  
✅ **Exercise 5:** Performed grouped aggregations  
✅ **Exercise 6:** Executed multiple join types  

---

### 🔑 Key Concepts:

| Concept | Function Used |
|---------|---------------|
| **Read CSV** | `spark.read.csv()` |
| **Filter** | `.filter()` or `.where()` |
| **Add Column** | `.withColumn()` |
| **Fill Missing** | `.fillna()` |
| **Aggregation** | `.groupBy().agg()` |
| **Joins** | `.join(how="inner/left/right")` |

---

### 💡 Join Types Recap:

- **Inner Join:** Only rows with matches in BOTH tables
- **Left Join:** ALL rows from left table + matches from right
- **Right Join:** ALL rows from right table + matches from left
- **Full Outer Join:** ALL rows from both tables (not shown)

---

## 🚀 Challenge Tasks (Optional):

1. **Calculate total revenue per customer**
   - Group by CustomerID
   - Sum TotalCost
   - Find top 3 customers by revenue

2. **Find the most popular product**
   - Group by Product
   - Sum Quantity
   - Order by total quantity descending

3. **Add discount column**
   - 10% discount if Quantity > 10
   - 5% discount if Quantity > 5
   - Calculate final price after discount

---

**Happy Practicing! 🎉**

In [ ]:
# Cleanup (optional): Stop Spark session
# spark.stop()
# print("✅ Spark session stopped")